In [1]:
import numpy as np

1*. Измените функцию calc_logloss так, чтобы нули по возможности не попадали в np.log.

In [5]:
def sigmoid(z):
    return 1/(1+np.exp(-z))

In [20]:
def calc_logloss(y, y_pred):
    res = []
    for index, element in enumerate(y_pred):
        if element == y[index]:
            res.append(0)
        elif element != 0 or element != 1:
            res.append(y[index]*np.log(element)+(1-y[index])*np.log(1-element))
        else:
            return np.inf
    return -np.mean(res)

2. Подберите аргументы функции eval_model для логистической регрессии таким образом, чтобы log loss был минимальным.

In [12]:
def eval_model(X, y, iterations, alpha=1e-4):
  np.random.seed(42)
  W = np.random.randn(X.shape[0])
  n = X.shape[1]
  for i in range(1, iterations+1):
    z = np.dot(W, X)
    y_pred = sigmoid(z)
    #err = calc_logloss(y, y_pred)
    W -= alpha * (1/n * np.dot((y_pred - y), X.T))
    #if i % (iterations / 10) == 0:
     # print(i, W, err)
  return W

In [8]:
X = np.array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
              [1, 1, 2, 1, 3, 0, 5, 10, 1, 2],
              [500, 700, 750, 600, 1450,
               800, 1500, 2000, 450, 1000],
              [1, 1, 2, 1, 2, 
               1, 3, 3, 1, 2]], dtype = np.float64)
y = np.array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1], dtype = np.float64)

In [9]:
def data_std(data):
    d_mean = data.mean(axis=1)
    d_std = data.std(axis=1)
    res = np.zeros(data.shape)
    for index, row in enumerate(data):
        res[index] = (row - d_mean[index])/d_std[index]
    return res

In [10]:
X_st = np.append(data_std(X[1:]), np.ones((1,X.shape[1])), axis=0)
X_st

array([[-0.57142857, -0.57142857, -0.21428571, -0.57142857,  0.14285714,
        -0.92857143,  0.85714286,  2.64285714, -0.57142857, -0.21428571],
       [-0.97958969, -0.56713087, -0.46401617, -0.77336028,  0.97958969,
        -0.36090146,  1.08270439,  2.11385144, -1.08270439,  0.05155735],
       [-0.89625816, -0.89625816,  0.38411064, -0.89625816,  0.38411064,
        -0.89625816,  1.66447944,  1.66447944, -0.89625816,  0.38411064],
       [ 1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
         1.        ,  1.        ,  1.        ,  1.        ,  1.        ]])

In [13]:
def param_search(X,y, iteration_list, alpha_list):
    best_err = np.inf
    best_params = []
    w = []
    err = 0
    y_pred = []
    for i in iteration_list:
        for alpha in alpha_list:
            w = eval_model(X,y,i, alpha)
            y_pred = sigmoid(np.dot(w,X))
            err = calc_logloss(y, y_pred)
            if err < best_err:
                best_err = err
                best_params = [i, alpha]
    return best_err, best_params

In [15]:
i_list = range(1000, 100001, 1000)
alpha_list = [1e-2, 1e-3,1e-4,1e-5]

In [21]:
best_err, best_params = param_search(X_st,y,i_list,alpha_list)
best_err, best_params

(0.1793792991927834, [100000, 0.01])

3. Создайте функцию calc_pred_proba, возвращающую предсказанную вероятность класса 1 (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred_proba).

In [22]:
W = eval_model(X_st, y, 100000, 0.01)

In [47]:
W

array([-6.05756455, -5.33520077, 13.26499674,  2.47794839])

In [23]:
def calc_pred_proba(W, X):
    return sigmoid(np.dot(W,X))

In [24]:
calc_pred_proba(W, X_st)

array([0.3267019 , 0.0509948 , 0.99998819, 0.13902456, 0.81479915,
       0.13463895, 0.99999875, 0.06123554, 0.45685695, 0.99981522])

4. Создайте функцию calc_pred, возвращающую предсказанный класс (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred).

In [25]:
def calc_pred(W, X):
    y_pred_proba = calc_pred_proba(W,X)
    res = []
    for y in y_pred_proba:
        if y < 0.5:
            res.append(0)
        else:
            res.append(1)
    return res

In [27]:
y_pred = calc_pred(W, X_st)
y_pred

[0, 0, 1, 0, 1, 0, 1, 0, 0, 1]

5. Посчитайте Accuracy, матрицу ошибок, точность и полноту, а также F1 score.

In [30]:
def accuracy(y, y_pred):
    res = []
    for real, pred in zip(y,y_pred):
        if real == pred:
            res.append(1)
        else:
            res.append(0)
    return np.mean(res)

In [31]:
accuracy(y,y_pred)

0.9

In [37]:
def err_matrix(y, y_pred):
    errors = np.zeros((2,2))
    for real, pred in zip(y, y_pred):
        if real == pred:
            if real == 1:
                errors[0,0] += 1
            else:
                errors[1,1] += 1
        else:
            if real == 1:
                errors[1,0] += 1
            else:
                errors[0,1] += 1
    return errors

In [39]:
errors = err_matrix(y, y_pred)
errors

array([[4., 0.],
       [1., 5.]])

In [40]:
precision = errors[0,0]/(errors[0,0]+errors[0,1])
precision

1.0

In [41]:
recall = errors[0,0]/(errors[0,0]+errors[1,0])
recall

0.8

In [42]:
F1 = 2*precision*recall/(precision+recall)
F1

0.888888888888889

6. Могла ли модель переобучиться? Почему?

Может. Не смотря на то что в нашей моделе всего 2 параметра, мы все еще можем получить переобученную и недообученную модели. Решением задачи может стать вектор w с большими весами, что является одним из признаков переобучения.

7*. Создайте функции eval_model_l1 и eval_model_l2 с применением L1 и L2 регуляризаций соответственно.

In [45]:
def eval_model_l1(X,y, iterations, alpha=1e-4, lambda_=1e-3):
    np.random.seed(42)
    W = np.random.randn(X.shape[0])
    for i in range(iterations):
        y_pred = sigmoid(W.dot(X))
        W -= alpha*(1/X.shape[1]*np.dot(y_pred-y, X.T)+lambda_*np.sign(W))
    return W

In [46]:
eval_model_l1(X_st, y, 100000, 0.01)

array([-5.672288  , -4.88154768, 12.28562137,  2.23540583])

In [48]:
def eval_model_l2(X, y, iterations, alpha=1e-4, lambda_=1e-3):
    np.random.seed(42)
    W = np.random.randn(X.shape[0])
    for i in range(iterations):
        y_pred = sigmoid(W.dot(X))
        W -= alpha*(1/X.shape[1]*np.dot(y_pred-y, X.T)+lambda_*W)
    return W

In [49]:
eval_model_l2(X_st, y, 100000, 0.01)

array([-4.23700165, -3.20835086,  8.65962171,  1.35448793])

In [50]:
W

array([-6.05756455, -5.33520077, 13.26499674,  2.47794839])

Как видно из значений W, регуляризация уменьшает значения. Не смотря на то, что мы выбрали достаточно небольшое значение lambda_, мы получили значения ощутимо меньше значений W без регуляризации. Это может значить что исходная модель переобучилась.